In [1]:
import keras
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
# Useful utilities
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
import hashlib as hasher
import datetime as date

class Block:
  def __init__(self, index, dataset, function, model, model_accuracy, previous_hash):
    self.index = index # Index in the blockchain
    self.timestamp = date.datetime.now()
    self.dataset = dataset
    self.function = function # The data preprocessing necessary to make the datatset operable.
    self.model = model
    self.model_accuracy = model_accuracy
    self.previous_hash = previous_hash
    self.hash = self.hash_block()
    
  def hash_block(self):
    sha = hasher.sha256()
    overall_data = str(self.index) + str(self.timestamp) + str(self.model_accuracy) + str(self.previous_hash)
    sha.update(overall_data.encode('utf-8'))
    return sha.hexdigest()

In [0]:
from keras.datasets import mnist 
dataset = mnist.load_data()

In [0]:
# The first block in a blockchain is called the genesis block.
def create_genesis_block(dataset, function):
  # Constructs a block with index zero and arbitrary "previous hash" parameter.
  return Block(0, dataset, function, "0", 0, "0")

In [0]:
def preprocessing(dataset):
    X_train = dataset[0][0]
    Y_train = dataset[0][1]
    X_test = dataset[1][0]
    Y_test = dataset[1][1]
    X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
    X_train = X_train.astype('float32')
    X_train /= 255
    X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
    X_test = X_test.astype('float32')
    X_test /= 255
    Y_train = np_utils.to_categorical(Y_train, 10)
    Y_test = np_utils.to_categorical(Y_test, 10)
    return X_train, Y_train, X_test, Y_test

In [0]:
first_block = create_genesis_block(dataset, preprocessing)
blockchain = []
blockchain.append(first_block)

In [7]:
len(blockchain)

1

In [0]:
def reset_chain(bc):
  d = bc[-1].dataset
  f = bc[-1].function
  bc = []
  first_block = create_genesis_block(d, f)
  bc.append(first_block)
  return bc

In [0]:
def new_block(last_block, model, model_accuracy):
  this_index = last_block.index + 1
  this_dataset = last_block.dataset
  this_function = last_block.function
  this_model = model
  this_model_accuracy = model_accuracy
  this_hash = last_block.hash
  return Block(this_index, this_dataset, this_function, this_model, this_model_accuracy, this_hash)

In [10]:
# Specify a new filepath if you already have a directory saved to the below name:

!pip install -q xlrd
!git clone https://github.com/Benendead/MNIST_Notebook MNIST
!ls MNIST

fatal: destination path 'MNIST' already exists and is not an empty directory.
 30_epochs  'Keras and CNNs (3).ipynb'	 README.md


In [11]:
def loadModel(library_path):
    """
    Loads a given Keras model from a filepath.
    """
    loaded_model = keras.models.load_model(library_path)
    print("Loaded model from path")
    return loaded_model
  
new_model = loadModel("MNIST/30_epochs")

Loaded model from path


In [12]:
type(new_model)

keras.models.Sequential

In [13]:
# Demonstration of turning hdf5 file into a String.

import h5py
file_name = ".h5"
new_model.save(file_name)
# type(pd.read_hdf(file_name))
# file_name = "/User/jameszhan/Downloads/weights.h5"
file = h5py.File(file_name, "r")
print("Keys: %s" % file.keys())
a_group_key = list(file.keys())[0]
data = list(file[a_group_key])
file.close()
print("\\".join(data))
s = '\\'.join(data)

Keys: KeysView(<HDF5 file ".h5" (mode r)>)
conv2d_1\conv2d_2\conv2d_3\dense_2\dense_3\dropout_1\dropout_2\dropout_3\flatten_1\max_pooling2d_1


In [0]:
import h5py

def commit_model(model, blockchain):
  """
  Attempts to commit a given Keras model into the blockchain.
    
  Parameters
  ----------
  model : Keras model
      The model to be uploaded.
  blockchain : Blockchain
      The blockchain for the model.
  """
  t = type(model)
  if(not (t == keras.models.Sequential or t == keras.models.Model)):   # Keras has two types of possible models.
    print("Model is not a valid Keras model.")
    return
      
  # First, perform proof of work:
  last_block = blockchain[-1]
  dataset = last_block.dataset
  f = last_block.function
  X_train, Y_train, X_test, Y_test = f(dataset)
    
  score = model.evaluate(X_test, Y_test, verbose = 1)
  acc = score[1]
  if (acc > last_block.model_accuracy):  
      
    # Save model to file:
    file_name = str(acc) + ".h5"
    model.save(file_name) # Keras handles this for us. 
    
    file = h5py.File(file_name, "r")
    # print("Keys: %s" % f.keys())
    a_group_key = list(file.keys())[0]
    data = list(file[a_group_key])
    file.close()
    
    print("The model's string:")
    print("\\".join(data))
    model_s = '\\'.join(data)
    
    block = new_block(last_block, model_s, acc)
    
    blockchain.append(block)
    print("Block committed successfully.")
    
  else:
    print("Block could not be committed, did not meet proof of work.")

In [15]:
blockchain = reset_chain(blockchain)
commit_model(new_model, blockchain)

10000/10000 [==============================] - 7s 653us/step
The model's string:
conv2d_1\conv2d_2\conv2d_3\dense_2\dense_3\dropout_1\dropout_2\dropout_3\flatten_1\max_pooling2d_1
Block committed successfully.


In [16]:
len(blockchain)

2

In [17]:
print(blockchain[1].model)

conv2d_1\conv2d_2\conv2d_3\dense_2\dense_3\dropout_1\dropout_2\dropout_3\flatten_1\max_pooling2d_1


In [0]:
def getCurrentBest(blockchain):
    """
    Downloads a given Keras model from the blockchain.
    The model is stored as an .h5 file.
    
    Parameters
    ----------
    blockchain : Blockchain
        The name of the desired model's blockchain.
    """
    
    #Load model from blockchain.
    loaded_model = blockchain[-1].model
    return loaded_model

In [24]:
model_block = getCurrentBest(blockchain)
model_block

'conv2d_1\\conv2d_2\\conv2d_3\\dense_2\\dense_3\\dropout_1\\dropout_2\\dropout_3\\flatten_1\\max_pooling2d_1'